# Convolutional Neural Network

### Importing the libraries

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
tf.__version__

'2.19.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [7]:
train_datagen = ImageDataGenerator(
        #rotation_range=40,
        #width_shift_range=0.2,
        #height_shift_range=0.2,
        rescale=1./255,            # diving each pixel by value 255 (as each pixel takes values between 0-255 so diving each by 255 to get value between 0 and 1) - Feature scaling
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        #fill_mode='nearest'
        )
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [9]:
test_datagen = ImageDataGenerator(rescale=1./255)            # applying feature scaling on the pixels of test set images ( no need to apply other rotation / flipping as its unseen data to avoid information leakage )
test_set = train_datagen.flow_from_directory('dataset/test_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [12]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [14]:
cnn.add(tf.keras.layers.Conv2D(filters =32, kernel_size = 3, activation = 'relu', input_shape = [64,64, 3]))

C:\Users\mohsi\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Step 2 - Pooling

In [16]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))

### Adding a second convolutional layer

In [18]:
cnn.add(tf.keras.layers.Conv2D(filters =32, kernel_size = 3, activation = 'relu')) # input_shape parameter is used when adding the first layer
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))

### Step 3 - Flattening

In [20]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [22]:
cnn.add(tf.keras.layers.Dense(units =128, activation= 'relu'))  # Units are neurons, larger number can give better result but can use any number

### Step 5 - Output Layer

In [24]:
cnn.add(tf.keras.layers.Dense(units=1, activation= 'sigmoid')) # Cat or Dog so using 1 as its Binary Classification also using sigmoid for same reason

## Part 3 - Training the CNN

### Compiling the CNN

In [27]:
cnn.compile(optimizer ='adam', loss ='binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [29]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)  # you can try different epochs for different result but this number works fine for this dataset

C:\Users\mohsi\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 29s 113ms/step - accuracy: 0.5407 - loss: 0.6891 - val_accuracy: 0.6810 - val_loss: 0.6026
Epoch 2/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 28s 113ms/step - accuracy: 0.6663 - loss: 0.6126 - val_accuracy: 0.7075 - val_loss: 0.5758
Epoch 3/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 30s 119ms/step - accuracy: 0.6977 - loss: 0.5668 - val_accuracy: 0.7340 - val_loss: 0.5375
Epoch 4/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 32s 126ms/step - accuracy: 0.7326 - loss: 0.5298 - val_accuracy: 0.7370 - val_loss: 0.5382
Epoch 5/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 29s 117ms/step - accuracy: 0.7571 - loss: 0.4949 - val_accuracy: 0.7660 - val_loss: 0.5001
Epoch 6/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 29s 116ms/step - accuracy: 0.7702 - loss: 0.4825 - val_accuracy: 0.7665 - val_loss: 0.5018
Epoch 7/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 29s 118ms/step - accuracy: 0.7652 - loss: 0.4701 - val_accuracy: 0.7575 - val_loss: 0.5152
Epoch 8/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 29s 116ms/step - accuracy: 0.7840 - loss: 0

## Part 4 - Making a single prediction

In [31]:
import numpy as np
from tensorflow.keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_5.jpg', target_size = (64,64))
test_image = image.img_to_array(test_image)  # predict function expect 2d Array so coverting img to array
test_image = np.expand_dims(test_image, axis = 0)  # It adds a new dimension at axis 0, turning this image into a batch of 1 image. (axis = 0, which means position is first ) as our model was trained on batches (batch = 32) but its a single image we wanna predict on
result = cnn.predict(test_image)
training_set.class_indices   # this line of code tells 1 corresponds to 'dog' and 0 corresponds to 'cat'
if result[0][0] ==1:        # first [0] in result[0][0] refers to batch and second [0] refers to element (as we have only 1 image (element) in a batch, it will be at first place and at first batch s0 [0][0]] )
  prediction = 'dog'
else:
  prediction = 'cat'


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step


In [32]:
print(prediction)

dog
